In [1]:
import pandas as pd
import json

### Get the Titles and cast

In [2]:
titlesandother = pd.read_csv("titlesandother.csv")

In [3]:
def explodetitleandcast(titlesandother):

    titleandcast = titlesandother[['title', 'cast']]
    titleandcast=titleandcast.fillna("")
    titleandcast=titleandcast.drop_duplicates()
    titleandcast=titleandcast.reset_index()
    titleandcast.drop(["index"], inplace=True, axis=1)
    titleandcast["cast"] = titleandcast["cast"].apply(lambda x :  x.split(",")  )
    titleandcastexploded = titleandcast.explode('cast')
    
    return titleandcastexploded

In [4]:
titleandcastexploded = explodetitleandcast(titlesandother)

In [5]:
titleandcastexploded.columns, len(titleandcastexploded)

(Index(['title', 'cast'], dtype='object'), 11939)

In [6]:
titleandcastexploded["cast"] = titleandcastexploded["cast"].apply(lambda x : x.strip())

In [7]:
thelistofactors = titleandcastexploded["cast"].unique().tolist()

### Get the actor count

In [8]:
# Opening JSON file
f = open('allactorsdict.json')

# returns JSON object as a dictionary
data = json.load(f)

# Closing file
f.close()

In [9]:
# thelistofactors = thelistofactors[38:50]

In [10]:
titleandcastexploded[titleandcastexploded["cast"]=="Laurence Fishburne"]

,title,cast
14,The Signal,Laurence Fishburne
126,The Ice Road,Laurence Fishburne
329,21,Laurence Fishburne
377,Mystic River,Laurence Fishburne
432,Teenage Mutant Ninja Turtles,Laurence Fishburne
648,The Matrix,Laurence Fishburne
859,The Matrix Reloaded,Laurence Fishburne
860,The Matrix Revolutions,Laurence Fishburne


In [11]:
titleandcastexploded[titleandcastexploded["cast"]=="Keanu Reeves"]

,title,cast
33,The Whole Truth,Keanu Reeves
396,Man of Tai Chi,Keanu Reeves
612,Knock Knock,Keanu Reeves
648,The Matrix,Keanu Reeves
669,Something's Gotta Give,Keanu Reeves
859,The Matrix Reloaded,Keanu Reeves
860,The Matrix Revolutions,Keanu Reeves


In [13]:
allcostararr = []
for actor in thelistofactors:
#     actor = "Taylor Swift"
    if(actor != ""):
        actorcostars = list(set(list(data[actor].keys())))
        actorcostars = list(set(actorcostars) - set(actor))
    #     len(actorcostars), actor

        ### Get an actor's movies

        thefilmsandshows = titleandcastexploded[titleandcastexploded["cast"]==actor]
        theactorsfilms = thefilmsandshows["title"].unique()

        ### Get the number of common movies and the common movies
        commonmoviesdict = {}
        for anactor in actorcostars:
            if(actor!=anactor):
                titles = titleandcastexploded[titleandcastexploded['cast']==anactor]["title"].unique().tolist()
                common_movies = list(set(titles) - (set(titles) - set(theactorsfilms)))
                num_movies = len(common_movies)

                dfactor = pd.DataFrame([[actor,anactor,num_movies,common_movies]], columns = ["actor", "co-star", "num movies costarring", "common_movies"])
                allcostararr.append(dfactor)

    #             commonmoviesdict[anactor] = [common_movies,len(common_movies)]

In [14]:
morecostars = pd.concat(allcostararr)

In [15]:
morecostars = morecostars.reset_index()

In [18]:
morecostars

,actor,co-star,num movies costarring,common_movies
0,Adjoa Andoh,Polly Walker,1,[Bridgerton]
1,Adjoa Andoh,Julie Andrews,1,[Bridgerton]
2,Adjoa Andoh,Lorraine Ashbourne,1,[Bridgerton]
3,Adjoa Andoh,Ben Miller,1,[Bridgerton]
4,Adjoa Andoh,RegÃ©-Jean Page,1,[Bridgerton]
...,...,...,...,...
124924,Hwang Bo-ra,Lee Ki-young,1,[Vagabond]
124925,Hwang Bo-ra,Bae Suzy,1,[Vagabond]
124926,Hwang Bo-ra,Lee Seung-gi,1,[Vagabond]
124927,Hwang Bo-ra,Moon Sung-keun,1,[Vagabond]


In [16]:
morecostars.drop(["index"], axis=1, inplace=True)

In [19]:
# morecostars = morecostars.drop_duplicates()

### Tag whether it is a franchise or not

In [34]:
from difflib import SequenceMatcher

# Utility function to compute similarity
def similar(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

In [67]:
def isitsequel(x):
    similarityls = []
    similarityval = False
    if(len(x) > 1):
        first = x[0]
        similarityls = []
        for othervalues in x[1:]:
            similarity = similar(first, othervalues)
            if(similarity > 0.55):
                similarityval = True 
            else:
                similarityval = False
            similarityls.append(similarityval)
            
    return similarityls

In [68]:
morecostars["is_franchise"] = morecostars["common_movies"].apply(lambda x : isitsequel(x))

In [69]:
morecostars.to_csv("morecostars.csv", index=False)